In [ ]:
!pip install scikit-surprise

In [ ]:
!pip install numpy==1.26.4
!pip install --no-cache-dir scikit-surprise

In [ ]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from surprise import accuracy
from sklearn.preprocessing import MinMaxScaler

# Load data
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/balanced_aqi_weather_suggestions.csv')

# Convert AQI category to numerical rating based on actual AQI ranges
aqi_rating_map = {
    'Good': 50,  # 0-50
    'Moderate': 100,  # 51-100
    'Unhealthy for Sensitive Groups': 150,  # 101-150
    'Unhealthy': 200,  # 151-200
    'Very Unhealthy': 300,  # 201-300
    'Hazardous': 500  # 301-500
}
df["Rating"] = df["AQI_Category"].map(aqi_rating_map)

# Remove the AQI_Category column
df.drop(columns=["AQI_Category"], inplace=True)

# Feature Engineering: Create interaction features between Age, Time_of_Day, AQI Rating, Temperature, and Precipitation
df["Age_Time_AQI_Interaction"] = df["Age"] * df["Time_of_Day"] * df["Rating"]
df["Temp_Precip_Interaction"] = df["Temperature"] * df["Precipitation"]  # Interaction of Temperature and Precipitation

# Normalize AQI Rating (scale between 0 and 5)
scaler = MinMaxScaler(feature_range=(0, 5))
df["Rating"] = scaler.fit_transform(df[["Rating"]])

# Rename columns to fit Surprise format
df.rename(columns={
    "Age": "User",  # Age is now numerical, used as 'User'
    "Time_of_Day": "Item"  # Time_of_Day is now numerical, used as 'Item'
}, inplace=True)

# Define Surprise Reader (adjust rating scale)
reader = Reader(rating_scale=(0, 5))  # Rating scale based on normalized AQI values

# Load dataset into Surprise
data = Dataset.load_from_df(df[["User", "Item", "Rating"]], reader)

# Split data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Hyperparameter Tuning using GridSearchCV
param_grid = {'n_factors': [50, 100, 150],
              'n_epochs': [10, 20],
              'lr_all': [0.002, 0.005, 0.01]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# Get the best parameters from GridSearchCV
best_params = gs.best_params['rmse']
print(f"Best Parameters: {best_params}")

# Train the best model using the optimal parameters
best_model = SVD(n_factors=best_params['n_factors'],
                 n_epochs=best_params['n_epochs'],
                 lr_all=best_params['lr_all'])
best_model.fit(trainset)

# Evaluate the best model using cross-validation
cross_validate(best_model, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

# Evaluate Model on the Test Set
predictions = best_model.test(testset)
print(f"RMSE: {accuracy.rmse(predictions)}")
print(f"MAE: {accuracy.mae(predictions)}")

# Function to Recommend the Best Activity (with AQI level, Temperature, and Precipitation consideration)
def recommend_activity(user_age, time_of_day, aqi_level, temperature, precipitation, df):
    activity_scores = {}

    for activity in df["Suggested_Activity"].unique():
        pred = best_model.predict(user_age, activity).est  # Use 'activity' as the item

        # Factor in AQI, Temperature, and Precipitation
        adjusted_pred = pred + (aqi_level / 500) + (temperature / 30) + (precipitation / 100)

        activity_scores[activity] = adjusted_pred

    # Get the best-rated activity
    best_activity = max(activity_scores, key=activity_scores.get)
    return best_activity

# Example usage of the function
recommended_activity = recommend_activity(user_age=25, time_of_day=2, aqi_level=120, temperature=25, precipitation=10, df=df)
print(f"Recommended Activity: {recommended_activity}")

Best Parameters: {'n_factors': 50, 'n_epochs': 10, 'lr_all': 0.002}
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.9183  1.9139  1.9097  1.9140  0.0035  
MAE (testset)     1.7048  1.6981  1.6917  1.6982  0.0054  
Fit time          0.41    0.41    0.42    0.42    0.00    
Test time         0.28    0.16    0.25    0.23    0.05    
RMSE: 1.8925
RMSE: 1.8924798687398061
MAE:  1.6761
MAE: 1.6760643696100477
Recommended Activity: Stay indoors


In [ ]:
# Example 1
user_age1 = 30
time_of_day1 = 9
aq_level1 = 50
temperature1 = 25  # Example temperature
precipitation1 = 5  # Example precipitation (in mm)

# Get Recommendation
recommended_activity1 = recommend_activity(user_age1, time_of_day1, aq_level1, temperature1, precipitation1, df)
print(f"Recommended Activity for Age {user_age1}, Time {time_of_day1}, AQI {aq_level1}: {recommended_activity1}")

# Example 2
user_age2 = 20
time_of_day2 = 12
aq_level2 = 200
temperature2 = 30
precipitation2 = 0

# Get Recommendation
recommended_activity2 = recommend_activity(user_age2, time_of_day2, aq_level2, temperature2, precipitation2, df)
print(f"Recommended Activity for Age {user_age2}, Time {time_of_day2}, AQI {aq_level2}: {recommended_activity2}")

# Example 3
user_age3 = 20
time_of_day3 = 17
aq_level3 = 450
temperature3 = 28
precipitation3 = 2

# Get Recommendation
recommended_activity3 = recommend_activity(user_age3, time_of_day3, aq_level3, temperature3, precipitation3, df)
print(f"Recommended Activity for Age {user_age3}, Time {time_of_day3}, AQI {aq_level3}: {recommended_activity3}")

# Example 4
user_age4 = 20
time_of_day4 = 12
aq_level4 = 15
temperature4 = 22
precipitation4 = 0

# Get Recommendation
recommended_activity4 = recommend_activity(user_age4, time_of_day4, aq_level4, temperature4, precipitation4, df)
print(f"Recommended Activity for Age {user_age4}, Time {time_of_day4}, AQI {aq_level4}: {recommended_activity4}")

# Example 5
user_age5 = 30
time_of_day5 = 21
aq_level5 = 10
temperature5 = 24
precipitation5 = 0

# Get Recommendation
recommended_activity5 = recommend_activity(user_age5, time_of_day5, aq_level5, temperature5, precipitation5, df)
print(f"Recommended Activity for Age {user_age5}, Time {time_of_day5}, AQI {aq_level5}: {recommended_activity5}")

# Example 6
user_age6 = 50
time_of_day6 = 6
aq_level6 = 300
temperature6 = 18
precipitation6 = 1

# Get Recommendation
recommended_activity6 = recommend_activity(user_age6, time_of_day6, aq_level6, temperature6, precipitation6, df)
print(f"Recommended Activity for Age {user_age6}, Time {time_of_day6}, AQI {aq_level6}: {recommended_activity6}")

# Example 7
user_age7 = 35
time_of_day7 = 14
aq_level7 = 55
temperature7 = 26
precipitation7 = 0

# Get Recommendation
recommended_activity7 = recommend_activity(user_age7, time_of_day7, aq_level7, temperature7, precipitation7, df)
print(f"Recommended Activity for Age {user_age7}, Time {time_of_day7}, AQI {aq_level7}: {recommended_activity7}")

# Example 8
user_age8 = 18
time_of_day8 = 23
aq_level8 = 500
temperature8 = 22
precipitation8 = 5

# Get Recommendation
recommended_activity8 = recommend_activity(user_age8, time_of_day8, aq_level8, temperature8, precipitation8, df)
print(f"Recommended Activity for Age {user_age8}, Time {time_of_day8}, AQI {aq_level8}: {recommended_activity8}")

# Example 9
user_age9 = 28
time_of_day9 = 16
aq_level9 = 90
temperature9 = 23
precipitation9 = 0

# Get Recommendation
recommended_activity9 = recommend_activity(user_age9, time_of_day9, aq_level9, temperature9, precipitation9, df)
print(f"Recommended Activity for Age {user_age9}, Time {time_of_day9}, AQI {aq_level9}: {recommended_activity9}")

# Example 10
user_age10 = 60
time_of_day10 = 7
aq_level10 = 15
temperature10 = 20
precipitation10 = 0

# Get Recommendation
recommended_activity10 = recommend_activity(user_age10, time_of_day10, aq_level10, temperature10, precipitation10, df)
print(f"Recommended Activity for Age {user_age10}, Time {time_of_day10}, AQI {aq_level10}: {recommended_activity10}")

# Example 11
user_age11 = 22
time_of_day11 = 19
aq_level11 = 175
temperature11 = 25
precipitation11 = 1

# Get Recommendation
recommended_activity11 = recommend_activity(user_age11, time_of_day11, aq_level11, temperature11, precipitation11, df)
print(f"Recommended Activity for Age {user_age11}, Time {time_of_day11}, AQI {aq_level11}: {recommended_activity11}")

# Example 12
user_age12 = 45
time_of_day12 = 10
aq_level12 = 42
temperature12 = 21
precipitation12 = 0

# Get Recommendation
recommended_activity12 = recommend_activity(user_age12, time_of_day12, aq_level12, temperature12, precipitation12, df)
print(f"Recommended Activity for Age {user_age12}, Time {time_of_day12}, AQI {aq_level12}: {recommended_activity12}")

# Example 13
user_age13 = 10
time_of_day13 = 15
aq_level13 = 30
temperature13 = 18
precipitation13 = 0

# Get Recommendation
recommended_activity13 = recommend_activity(user_age13, time_of_day13, aq_level13, temperature13, precipitation13, df)
print(f"Recommended Activity for Age {user_age13}, Time {time_of_day13}, AQI {aq_level13}: {recommended_activity13}")

# Example 14
user_age14 = 25
time_of_day14 = 22
aq_level14 = 75
temperature14 = 23
precipitation14 = 0

# Get Recommendation
recommended_activity14 = recommend_activity(user_age14, time_of_day14, aq_level14, temperature14, precipitation14, df)
print(f"Recommended Activity for Age {user_age14}, Time {time_of_day14}, AQI {aq_level14}: {recommended_activity14}")

# Example 15
user_age15 = 55
time_of_day15 = 5
aq_level15 = 125
temperature15 = 19
precipitation15 = 0

# Get Recommendation
recommended_activity15 = recommend_activity(user_age15, time_of_day15, aq_level15, temperature15, precipitation15, df)
print(f"Recommended Activity for Age {user_age15}, Time {time_of_day15}, AQI {aq_level15}: {recommended_activity15}")

# Example 16
user_age16 = 40
time_of_day16 = 17
aq_level16 = 220
temperature16 = 27
precipitation16 = 1

# Get Recommendation
recommended_activity16 = recommend_activity(user_age16, time_of_day16, aq_level16, temperature16, precipitation16, df)
print(f"Recommended Activity for Age {user_age16}, Time {time_of_day16}, AQI {aq_level16}: {recommended_activity16}")

# Example 17
user_age17 = 70
time_of_day17 = 11
aq_level17 = 400
temperature17 = 21
precipitation17 = 0

# Get Recommendation
recommended_activity17 = recommend_activity(user_age17, time_of_day17, aq_level17, temperature17, precipitation17, df)
print(f"Recommended Activity for Age {user_age17}, Time {time_of_day17}, AQI {aq_level17}: {recommended_activity17}")

# Example 18
user_age18 = 20
time_of_day18 = 14
aq_level18 = 2
temperature18 = 25
precipitation18 = 0

# Get Recommendation
recommended_activity18 = recommend_activity(user_age18, time_of_day18, aq_level18, temperature18, precipitation18, df)
print(f"Recommended Activity for Age {user_age18}, Time {time_of_day18}, AQI {aq_level18}: {recommended_activity18}")

Recommended Activity for Age 30, Time 9, AQI 50: Stay indoors
Recommended Activity for Age 20, Time 12, AQI 200: Stay indoors
Recommended Activity for Age 20, Time 17, AQI 450: Stay indoors
Recommended Activity for Age 20, Time 12, AQI 15: Stay indoors
Recommended Activity for Age 30, Time 21, AQI 10: Stay indoors
Recommended Activity for Age 50, Time 6, AQI 300: Stay indoors
Recommended Activity for Age 35, Time 14, AQI 55: Stay indoors
Recommended Activity for Age 18, Time 23, AQI 500: Stay indoors
Recommended Activity for Age 28, Time 16, AQI 90: Stay indoors
Recommended Activity for Age 60, Time 7, AQI 15: Stay indoors
Recommended Activity for Age 22, Time 19, AQI 175: Stay indoors
Recommended Activity for Age 45, Time 10, AQI 42: Stay indoors
Recommended Activity for Age 10, Time 15, AQI 30: Stay indoors
Recommended Activity for Age 25, Time 22, AQI 75: Stay indoors
Recommended Activity for Age 55, Time 5, AQI 125: Stay indoors
Recommended Activity for Age 40, Time 17, AQI 220: St